# Imports

In [ ]:
!pip install transformers
!pip install Levenshtein
!pip install word2number
!pip install pulp

In [2]:
import warnings
import torch
import json
from transformers import BertTokenizer, BertForSequenceClassification, BertForTokenClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import requests
import json
import sys
import Levenshtein
from word2number import w2n
from sklearn.neighbors import NearestNeighbors
import pulp
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')

# Mounting

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Models

In [4]:
intent_model_path = '/content/drive/MyDrive/23-24/CS 4701/Intent Trained Model'
NER_model_path = '/content/drive/MyDrive/23-24/CS 4701/NER Trained Model'
intent_model = BertForSequenceClassification.from_pretrained(intent_model_path)
intent_tokenizer = BertTokenizer.from_pretrained(intent_model_path)
NER_model = BertForTokenClassification.from_pretrained(NER_model_path)
NER_tokenizer = BertTokenizer.from_pretrained(NER_model_path)

In [5]:
ner_label_map = {'O': 0,
             'B-MealType': 1,
             'I-MealType': 2,
             'B-Food': 3,
             'I-Food': 4,
             'B-Dietary': 5,
             'I-Dietary': 6,
             'B-Nutrient': 7,
             'I-Nutrient': 8,
             'B-Muscle': 9,
             'I-Muscle': 10,
             'B-MuscleGroups': 11,
             'I-MuscleGroups': 12,
             'B-Time': 13,
             'I-Time': 14,
             'B-Workout': 15,
             'I-Workout': 16,
             'B-Goal': 17,
             'I-Goal': 18,
             'B-Age': 19,
             "B-Equipment": 20,
             "B-Number":21,
             "I-Number":22,
             "B-Weight": 23,
             "I-Weight": 24,
             "B-Duration":25,
             "I-Duration":26}
id_to_response = {0: "CreateNutritionPlan", 1: "CreateWorkoutPlan", 2: "RecommendMeal", 3: "RecommendExercise", 4: "QueryFood", 5: "QueryExercise", 6: "Unrelated"}
# Function to predict intent
def predict_intent(sentence, model, tokenizer):
  model.eval()
  inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)
  with torch.no_grad():
      logits = model(**inputs).logits
  predicted_label_id = logits.argmax().item()
  try:
      return id_to_response[predicted_label_id]
  except:
      return "Unrelated"
# Function to predict entities (same as in your ner.py)
def predict_entities(sentence, model, tokenizer, label_map):
    model.eval()  # Put model in evaluation mode

    # Tokenize the sentence and align with the BERT tokens
    inputs = tokenizer(sentence, truncation=True, padding=True, return_tensors="pt")

    # Move the inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Predict
    with torch.no_grad():
        outputs = model(**inputs).logits

    # Find the predicted labels
    predictions = torch.argmax(outputs, dim=2)

    # Convert the predictions to label names
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0].to('cpu'))
    label_indices = predictions[0].to('cpu').numpy()
    labels = [list(label_map.keys())[list(label_map.values()).index(i)] for i in label_indices]

    # Pair each token with its label
    token_label_pairs = list(zip(tokens, labels))
    return token_label_pairs
# Function to clean predictions (same as in your ner.py)
def clean_predictions(token_label_pairs):
    # Filter out special tokens and subword tokens
    return [(token, label) for token, label in token_label_pairs if token not in ["[CLS]", "[SEP]", "[PAD]"] and not token.startswith("##")]
def align_words_with_entities(user_input, cleaned_entities):
    words = user_input.split()  # Split the user input into words
    aligned_entities = []
    word_index = 0
    for token, label in cleaned_entities:
        if token.startswith("##"):
            # Merge with the previous word
            aligned_entities[-1] = (aligned_entities[-1][0] + token[2:], label)
        else:
            # Align with the current word
            if word_index < len(words):
                aligned_entities.append((words[word_index], label))
                word_index += 1

    return aligned_entities

# QueryFood

In [6]:
class FoodNutrient:
    def __init__(self):
        self.Calcium = None
        self.Iron = None
        self.Magnesium = None
        self.Potassium = None
        self.Zinc = None
        self.Selenium = None
        self.Phosphorus = None
        self.Vitamin_A = None
        self.Vitamin_B12 = None
        self.Vitamin_B6 = None
        self.Vitamin_C = None
        self.Vitamin_D = None
        self.Vitamin_D4 = None
        self.Vitamin_E = None
        self.Vitamin_K = None
        self.Carbohydrates = None
        self.Protein = None
        self.Calories = None
        self.Fats = None
        self.Saturated_Fats = None
        self.Trans_Fats = None
        self.Sodium = None
        self.Sugar = None

    def get_vector(self):
      protein = self.Protein[1]
      calories = self.Calories[1]
      carbs = self.Carbohydrates[1]
      fats = self.Fats[1]
      return [protein * 4/ calories, carbs * 4/ calories, fats * 9 / calories]


    def update_nutrient(self, nutrient_name, amount, unitName):
        if hasattr(self, nutrient_name):
            if (getattr(self, nutrient_name)):
                return
            setattr(self, nutrient_name, [nutrient_name, amount, unitName])

    def get_attributes(self):
        return [attr for attr in dir(self) if not attr.startswith('__') and not callable(getattr(self, attr)) and getattr(self, attr) is not None]

    def pretty_print(self, food_name, quant_s, available_nutrients):
      available_nutrient_names = [nutrient_li[0] for nutrient_li in available_nutrients]
      ordered_nutrients = ["Calories", "Protein", "Fats", "Carbohydrates",
                             "Calcium", "Iron", "Magnesium", "Potassium",
                             "Zinc", "Selenium", "Phosphorus", "Vitamin_A",
                             "Vitamin_B12", "Vitamin_B6", "Vitamin_C",
                             "Vitamin_D", "Vitamin_D4", "Vitamin_E",
                             "Vitamin_K", "Saturated_Fats", "Trans_Fats",
                             "Sodium", "Sugar"]
      header =f"Nutrient values ({quant_s}) of {food_name}:"
      print("-" * len(header))
      print(header)
      print("-" * len(header))
      for nutrient in ordered_nutrients:
        try:
          index = available_nutrient_names.index(nutrient)
          nutrient_info = available_nutrients[index]
          print(f"{nutrient_info[0]:15}: {int(nutrient_info[1]):>8}{nutrient_info[2]}")
        except:
          continue
      print("-" * len(header))



    def print_nutrients(self, food_name):
        # Define the order of nutrients
        ordered_nutrients = ["Calories", "Protein", "Fats", "Carbohydrates",
                             "Calcium", "Iron", "Magnesium", "Potassium",
                             "Zinc", "Selenium", "Phosphorus", "Vitamin_A",
                             "Vitamin_B12", "Vitamin_B6", "Vitamin_C",
                             "Vitamin_D", "Vitamin_D4", "Vitamin_E",
                             "Vitamin_K", "Saturated_Fats", "Trans_Fats",
                             "Sodium", "Sugar"]

        # Create a header for the table
        header = f"Nutrient values (100 g) of {food_name}"
        print("-" * len(header))
        print(header)
        print("-" * len(header))

        # Print each nutrient in order
        for nutrient in ordered_nutrients:
            value = getattr(self, nutrient, None)
            if value:
                print(f"{nutrient:15}: {value[1]:>8}{value[2]}")

        # End the table with a line
        print("-" * len(header))

nutrient_mapping = {
    "Calcium": [1087, 1237, 1239, 301],
    "Iron": [1089, 1238, 1141, 1240, 1142, 303],
    "Magnesium": [1090,304],
    "Potassium": [1092,305],
    "Zinc": [1095,309],
    "Selenium": [1103,317],
    "Phosphorus": [1091,305],
    "Vitamin_A": [2067,1104,1106,1156,960,318,320,392],
    "Vitamin_B12": [1178,1246,1252,418,578,588],
    "Vitamin_B6": [1175,1174,1172,1173,1171,415,414,412,413,411],
    "Vitamin_C": [1241,1164,1247,1163,1162,571,403,581,402,401],
    "Vitamin_D": [1114, 1110, 328, 324],
    "Vitamin_D4": [2059],
    "Vitamin_E": [1158,2068,1109,1124,1242,1248,394,959,323,340,573,583],
    "Vitamin_K": [1184,1183,1185, 429, 428, 430],
    "Carbohydrates": [2039, 1050, 1072, 1005, 205],
    "Protein": [1003, 1053, 203],
    "Calories": [1008, 2047, 2048, 208],
    "Fats": [1004, 204],
    "Saturated_Fats": [1258, 1326, 606, 690],
    "Trans_Fats" : [1257, 605],
    "Sodium" : [1093, 307],
    "Sugar": [1063, 269, 2000, 1235, 1236]
}

Macros_li = ["Protein", "Carbohydrates", "Fats", "Calories"]
Vitamins = ["Vitamin_A", "Vitamin_B12", "Vitamin_B6", "Vitamin_C", "Vitamin_D", "Vitamin_D4", "Vitamin_E", "Vitamin_K"]
Minerals = ["Calcium", "Iron", "Magnesium", "Potassium", "Zinc", "Selenium", "Phosphorus"]
Electrolytes = ["Sodium", "Potassium", "Magnesium"]
Nutrition = [
    "Calcium", "Iron", "Magnesium", "Potassium", "Zinc",
    "Selenium", "Phosphorus", "Vitamin_A", "Vitamin_B12",
    "Vitamin_B6", "Vitamin_C", "Vitamin_D", "Vitamin_D4",
    "Vitamin_E", "Vitamin_K", "Carbohydrates", "Protein",
    "Calories", "Fats", "Saturated_Fats", "Trans_Fats",
    "Sodium", "Sugar"
]
matching_name = {
    "Calcium": ["Calcium"],
    "Iron": ["Iron"],
    "Magnesium": ["Magnesium"],
    "Potassium": ["Potassium"],
    "Zinc": ["Zinc"],
    "Selenium": ["Selenium"],
    "Phosphorus": ["Phosphorus"],
    "Vitamin A": ["Vitamin_A"],
    "Vitamin B12": ["Vitamin_B12"],
    "Vitamin B6": ["Vitamin_B6"],
    "Vitamin C": ["Vitamin_C"],
    "Vitamin D": ["Vitamin_D"],
    "Vitamin D4": ["Vitamin_D4"],
    "Vitamin E": ["Vitamin_E"],
    "Vitamin K": ["Vitamin_K"],
    "Carbohydrates": ["Carbohydrates"],
    "Protein": ["Protein"],
    "Calories": ["Calories"],
    "Fats": ["Fats", "Saturated_Fats", "Trans_Fats"],
    "Saturated Fats": ["Saturated_Fats"],
    "Trans Fats": ["Trans_Fats"],
    "Sodium": ["Sodium"],
    "Sugar": ["Sugar"],
    "Macros": Macros_li,
    "Macronutrients": Macros_li,
    "Vitamins": Vitamins,
    "Vits": Vitamins,
    "Vitamin": Vitamins,
    "Minerals": Minerals,
    "Micros": Minerals,
    "Micronutrients": Minerals,
    "Electrolytes": Electrolytes,
    "Nutrition" : Nutrition,
    "Information" : Nutrition,
    "Nutritional Information" : Nutrition
}

def find_closest_match(query, choices, threshold=0.8):
    best_match = None
    highest_similarity = 0

    for choice in choices:
        similarity = Levenshtein.ratio(query.lower(), choice.lower())
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = choice

    if highest_similarity >= threshold:
        return best_match
    else:
        return None  # or return an appropriate message


api_key = 'b8goQINekuJLoWVOfycPYltP6B6ZCeuckXkedZwb'

def get_food_nutrients(food_nutrients, abridged):
    food_nutrient_obj = FoodNutrient()
    for nutrient in food_nutrients:
        nutrient_dict = nutrient.get('nutrient', nutrient)
        if nutrient_dict:
            nutrient_name = nutrient_dict.get('name', "N/A")
            amount = nutrient.get('amount', 'N/A')  # Default to 'N/A' if amount is not present
            unitName = nutrient_dict.get('unitName', "N/A")
            nutrient_id_str = nutrient_dict.get('number' if abridged else 'id', None)
            try:
                nutrient_id = int(nutrient_id_str)  # Convert to integer
                for key, ids in nutrient_mapping.items():
                    if nutrient_id in ids:
                        food_nutrient_obj.update_nutrient(key, amount, unitName)
                        break
                #print(f"{nutrient_name}: {amount} {unitName}")
            except ValueError:
                print(f"Invalid nutrient ID format: {nutrient_id_str}")
        else:
            print("Cant find nutrient")
    #food_nutrient_obj.print_nutrients()
    return food_nutrient_obj


def call_response(base_url, params, abridged=False):
    response = requests.get(base_url, params)
    if response.status_code == 200:
        data = response.json()
        food_nutrients = data.get('foodNutrients', [])
        if food_nutrients:
            #print(data.get('description', "N/A"))
            return get_food_nutrients(food_nutrients, abridged)  # Return the FoodNutrient object
        else:
            return None
    else:
        print("Server error", response.url)
        return None


def get_food_data(fdc_id):
    base_url = f'https://api.nal.usda.gov/fdc/v1/food/{fdc_id}'
    params = {'api_key': api_key, 'format': 'full'}
    food_nutrient_obj = call_response(base_url, params)
    if food_nutrient_obj:
        return food_nutrient_obj

    params = {'api_key': api_key, 'format': 'abridged'}
    return call_response(base_url, params, abridged=True)


def get_food_id(food_name):
    base_url = f'https://api.nal.usda.gov/fdc/v1/foods/search'
    params = {'api_key': api_key,
              'query': food_name,
              'dataType': 'Foundation,Branded',
              'sortBy': 'dataType.keyword',
              'requireAllWords': 'true'}

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'foods' in data and len(data['foods']) > 0:
            fdc_id = data['foods'][0].get("fdcId", None)  # Adjusted to extract the first fdcId
            return fdc_id
        else:
            print(response.url)
            return None
    else:
        return None
Kilograms = "kg"
Pounds = "lb"
Grams = "g"
Ounces = "oz"
weight_mapping ={
    Kilograms: ["KG", "Kilos", "Kilograms", "Kilogram", "Kilog"],
    Pounds: ["LB", "Pounds"],
    Grams: ["Grams", "G"],
    Ounces: ["OZ", "Ounces", "Ounce"]
}

def find_closest_weight_unit(query, weight_mapping, threshold=0.8):
    best_match_unit = None
    highest_similarity = 0

    for unit, expressions in weight_mapping.items():
        for expression in expressions:
            similarity = Levenshtein.ratio(query.lower(), expression.lower())
            if similarity > highest_similarity:
                highest_similarity = similarity
                best_match_unit = unit

    if highest_similarity >= threshold:
        return best_match_unit
    else:
        return None  # or return an appropriate message
def get_conversion_factor(unit):
    conversion_factors = {
        'kg': 1000,    # 1 kg = 1000 g
        'lb': 453.592, # 1 lb = 453.592 g
        'g': 1,        # 1 g = 1 g
        'oz': 28.3495  # 1 oz = 28.3495 g
    }
    return conversion_factors.get(unit, None)
def scale_nutrients(nutrients, quantity, unit):
  matching_unit = find_closest_weight_unit(unit, weight_mapping)
  if matching_unit is None:
      print("Invalid unit provided.")
      return nutrients, False

  conversion_factor = get_conversion_factor(matching_unit)
  if conversion_factor is None:
      print("Conversion factor not found for the unit.")
      return nutrients, False

  scaled_quantity = float(quantity) * conversion_factor / 100  # Convert to the scale of 100g
  scaled_nutrients = []
  for nutrient in nutrients:
      nutrient_name, nutrient_value, nutrient_unit = nutrient
      scaled_value = nutrient_value * scaled_quantity
      scaled_nutrients.append([nutrient_name, scaled_value, nutrient_unit.lower()])
  return scaled_nutrients, True

def extract_number_and_weight(cleaned_predictions):
    number_entity = ""
    weight_entity = ""

    for i, (token, label) in enumerate(cleaned_predictions):
        # Extract number entity
        if label in ['B-Number', 'I-Number']:
            try:
                number_entity = str(w2n.word_to_num(token.lower()))
            except ValueError:
                number_entity = token

        if label == 'B-Weight' and (i > 0 and (cleaned_predictions[i - 1][1] in ['B-Number', 'I-Number'] or number_entity)):
            weight_entity = token
            j = i + 1
            while j < len(cleaned_predictions) and cleaned_predictions[j][1] == "I-Weight":
                weight_entity += " " + cleaned_predictions[j][0]
                j += 1
            break

    return number_entity, weight_entity
def extract_nutrient_entity(cleaned_predictions):
    nutrient_entity = []
    for token, label in cleaned_predictions:
        if label in ['B-Nutrient', 'I-Nutrient']:
            nutrient_entity.append(token)
    return " ".join(nutrient_entity)
def extract_food_entity(cleaned_predictions):
    food_entity = []
    for token, label in cleaned_predictions:
        if label in ['B-Food', 'I-Food']:
            food_entity.append(token)
    return " ".join(food_entity)

def normalize_vector(vector):
    norm = sum(vector)
    return [x / norm for x in vector]

def queryFood_main(food_name, nutrient_name, quantity, unit, user_profile):
  food_id = get_food_id(food_name)
  if food_id:
    food_nutrient_obj = get_food_data(food_id)
    if food_nutrient_obj:
      available_nutrients = []
      mapped_nutrients = []
      if (nutrient_name == ""):
        mapped_nutrients = food_nutrient_obj.get_attributes()
      else:
        closest_match = find_closest_match(nutrient_name, matching_name.keys())
        mapped_nutrients = matching_name.get(closest_match, [closest_match])
        if mapped_nutrients == [None]:
          mapped_nutrients = food_nutrient_obj.get_attributes()
      for mapped_nutrient in mapped_nutrients:
        if hasattr(food_nutrient_obj, mapped_nutrient):
          nutrient_value = getattr(food_nutrient_obj, mapped_nutrient)
          if nutrient_value:
            available_nutrients.append(nutrient_value)
      res = False
      if quantity != "" and unit != "":
        available_nutrients, res = scale_nutrients(available_nutrients, quantity, unit)
      quant_s = "100 g" if not res else f"{quantity} {unit}"
      food_nutrient_obj.pretty_print(food_name, quant_s, available_nutrients)


# QueryExercise


In [7]:
def extract_workout_entity(cleaned_predictions):
  workout_entity = []
  for token, label in cleaned_predictions:
    if label in ['B-Workout', 'I-Workout']:
      workout_entity.append(token)
  return " ".join(workout_entity)

In [8]:
data_path = "/content/drive/MyDrive/23-24/CS 4701/gym_exercises.csv"

df = pd.read_csv(data_path)

matching_muscles = {
    "bench": "bench press",
    "squat": "barbell front squat",
    "squatting": "barbell front squat"
}

def replace_db_with_dumbbell_if_present(text):
    if "db" in text:
        return text.replace("db", "dumbbell")
    return text

def _find_closest_match(query, df, threshold=0.4):
    best_match = None
    highest_similarity = 0

    modified_query = replace_db_with_dumbbell_if_present(query)

    try:
      expression = matching_muscles[modified_query]
    except:
      expression = modified_query

    for index, row in df.iterrows():
        similarity = Levenshtein.ratio(expression, row[0])
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = row[0]

    result = df.loc[df["exercise_name"] == best_match, "muscle_groups"].iloc[0]

    if highest_similarity >= threshold:
        return result, best_match
    else:
        return None

def query_exercise(exercise):
    muscle_groups, best_match = _find_closest_match(exercise.lower(), df)
    if muscle_groups is not None:
        # Header for exercise
        header = f"Details of '{best_match}' Exercise:"
        print("-" * len(header))
        print(header)
        print("-" * len(header))

        # Printing muscle groups targeted
        print(f"Targets Muscles: {muscle_groups}")

        # Ending line separator
        print("-" * len(header))
    else:
        print("Bad query, try using another exercise name.")

# CreateNutritionPlan

In [9]:
data_path = "/content/drive/MyDrive/23-24/CS 4701/nutrition.csv"
class CustomNearestNeighborsDataFrame:
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
        self.model = NearestNeighbors(n_neighbors=self.n_neighbors)

    def fit(self, df):
        self.data = df.iloc[:, 1:5]
        self.means = self.data.mean()
        self.std = self.data.std()
        self.data = (self.data-self.data.mean())/self.data.std()
        self.names = df.iloc[:, 0]
        self.model.fit(self.data)

    def find_nearest(self, point):
        point = (point - self.means) / self.std
        distances, indices = self.model.kneighbors([point])
        return self.names.iloc[indices[0]].tolist()


def create_nutrition_plan(daily_cals, daily_protein, daily_carbs, daily_fats):
    b_macros = [0.2*daily_cals, 0.2*daily_protein, 0.2*daily_fats, 0.2*daily_carbs]
    ld_macros = [0.8*daily_cals, 0.8*daily_protein, 0.8*daily_fats, 0.8*daily_carbs]
    s_macros = [0.1*daily_cals, 0.1*daily_protein, 0.1*daily_fats, 0.1*daily_carbs]

    df = pd.read_csv(data_path)
    df = df.dropna()
    df = df[df['Meal (1=B, 2=L/D, 3=S)'] == 1]
    df = df.iloc[:, [1, 4, 5, 6, 7]]
    custom_nn_df = CustomNearestNeighborsDataFrame(n_neighbors=5)
    custom_nn_df.fit(df)
    b_list = custom_nn_df.find_nearest(b_macros)

    df = pd.read_csv(data_path)
    df = df.dropna()
    df = df[df['Meal (1=B, 2=L/D, 3=S)'] == 2]
    df = df.iloc[:, [1, 4, 5, 6, 7]]
    custom_nn_df = CustomNearestNeighborsDataFrame(n_neighbors=10)
    custom_nn_df.fit(df)
    ld_list = custom_nn_df.find_nearest(ld_macros)

    df = pd.read_csv(data_path)
    df = df.dropna()
    df = df[df['Meal (1=B, 2=L/D, 3=S)'] == 3]
    df = df.iloc[:, [1, 4, 5, 6, 7]]
    custom_nn_df = CustomNearestNeighborsDataFrame(n_neighbors=5)
    custom_nn_df.fit(df)
    s_list = custom_nn_df.find_nearest(s_macros)

    return b_list, ld_list, s_list

def create_lp_model(user_profile, b_list, l_list, d_list, s_list):

    prob = pulp.LpProblem("MealPlanOptimization", pulp.LpMinimize)

    all_meals = b_list + l_list + d_list + s_list
    meal_vars = [pulp.LpVariable(f"meal_{i}", cat='Binary') for i in range(len(all_meals))]

    cal_diff_pos = pulp.LpVariable("cal_diff_pos", lowBound=0)
    cal_diff_neg = pulp.LpVariable("cal_diff_neg", lowBound=0)
    prot_diff_pos = pulp.LpVariable("prot_diff_pos", lowBound=0)
    prot_diff_neg = pulp.LpVariable("prot_diff_neg", lowBound=0)
    fats_diff_pos = pulp.LpVariable("fats_diff_pos", lowBound=0)
    fats_diff_neg = pulp.LpVariable("fats_diff_neg", lowBound=0)
    carbs_diff_pos = pulp.LpVariable("carbs_diff_pos", lowBound=0)
    carbs_diff_neg = pulp.LpVariable("carbs_diff_neg", lowBound=0)

    prob += pulp.lpSum(meal_vars[:len(b_list)]) == 1
    prob += pulp.lpSum(meal_vars[len(b_list):len(b_list)+len(l_list)]) == 1
    prob += pulp.lpSum(meal_vars[len(b_list)+len(l_list):len(b_list)+len(l_list)+len(d_list)]) == 1
    prob += pulp.lpSum(meal_vars[len(b_list)+len(l_list)+len(d_list):]) == 1

    total_calories = pulp.lpSum(all_meals[i][4] * meal_vars[i] for i in range(len(all_meals)))
    total_protein = pulp.lpSum(all_meals[i][5] * meal_vars[i] for i in range(len(all_meals)))
    total_fats = pulp.lpSum(all_meals[i][6] * meal_vars[i] for i in range(len(all_meals)))
    total_carbs = pulp.lpSum(all_meals[i][7] * meal_vars[i] for i in range(len(all_meals)))

    prob += total_calories - user_profile.calories == cal_diff_pos - cal_diff_neg
    prob += total_protein - user_profile.macros.protein_grams == prot_diff_pos - prot_diff_neg
    prob += total_fats - user_profile.macros.fats_grams == fats_diff_pos - fats_diff_neg
    prob += total_carbs - user_profile.macros.carbs_grams == carbs_diff_pos - carbs_diff_neg

    prob += cal_diff_pos + cal_diff_neg + prot_diff_pos + prot_diff_neg + fats_diff_pos + fats_diff_neg + carbs_diff_pos + carbs_diff_neg


    prob.solve()

    chosen_meals = []
    b_chosen = [all_meals[i] for i in range(len(b_list)) if meal_vars[i].varValue > 0.5]
    l_chosen = [all_meals[i] for i in range(len(b_list), len(b_list) + len(l_list)) if meal_vars[i].varValue > 0.5]
    d_chosen = [all_meals[i] for i in range(len(b_list) + len(l_list), len(b_list) + len(l_list) + len(d_list)) if meal_vars[i].varValue > 0.5]
    s_chosen = [all_meals[i] for i in range(len(b_list) + len(l_list) + len(d_list), len(all_meals)) if meal_vars[i].varValue > 0.5]

    chosen_meals.extend(b_chosen + l_chosen + d_chosen + s_chosen)

    return chosen_meals

def print_nutrition(user_profile):
    nutrition = user_profile.get_nutrition()
    header = "Your Daily Calories and Macros"
    separator = "-" * (len(header) + 5)

    # Table-like formatting
    print(separator)
    print(header)
    print(separator)
    print(f"{'Daily Calories':25s}: {nutrition[0]:>10.2f}")
    print(f"{'Daily Protein (g)':25s}: {nutrition[1].protein_grams:>10.2f}")
    print(f"{'Daily Carbohydrates (g)':25s}: {nutrition[1].carbs_grams:>10.2f}")
    print(f"{'Daily Fats (g)':25s}: {nutrition[1].fats_grams:>10.2f}")
    print(separator)

def multi_servings(meal_list):
    extended_list = []
    for meal in meal_list:
        single_serving = meal.copy()
        single_serving.append('single')  # Add identifier for single serving
        extended_list.append(single_serving)

        double_serving = meal.copy()
        triple_serving = meal.copy()
        for i in range(4, 8):  # Double the nutritional content
            double_serving[i] *= 2
            triple_serving[i] *= 3
        double_serving.append('double')  # Add identifier for double serving
        triple_serving.append('triple')
        extended_list.append(double_serving)
        extended_list.append(triple_serving)


    return extended_list

def get_recipe_info(list):
  df = pd.read_csv(data_path)
  filtered_df = df[df['Name'].isin(list)]
  return filtered_df.values.tolist()
def pretty_print_createNutritionPlan(chosen_meals):
    meal_categories = ["Breakfast", "Lunch", "Dinner", "Snack"]

    # Initialize total nutrition counters
    total_calories = 0
    total_protein = 0
    total_fats = 0
    total_carbs = 0

    print("\nMeal Plan")
    table_width = 140
    print("-" * table_width)
    print("{:<12} {:<50} {:<12} {:<10} {:<12} {:<10} {:<10} {:<10} {:<10}".format(
        "Meal Type", "Recipe", "Time (mins)", "Servings", "Calories", "Protein (g)", "Fats (g)", "Carbs (g)", "Recipe Link"))
    print("-" * table_width)

    for meal, category in zip(chosen_meals, meal_categories):
        name = meal[1]
        # Truncate and concatenate long names
        if len(name) > 47:
            name = name[:47] + '...'

        time, servings, calories, protein, fats, carbs, link = meal[2], ("Double" if meal[-1] == 'double' else ("Single" if meal[-1] == 'single' else "Triple")), meal[4], meal[5], meal[6], meal[7], meal[9]

        # Update total nutrition counters
        total_calories += calories
        total_protein += protein
        total_fats += fats
        total_carbs += carbs

        print("{:<12} {:<50} {:<12} {:<10} {:<12} {:<10} {:<10} {:<10} {:<10}".format(
              category, name, time, servings, calories, protein, fats, carbs, link))

    print("-" * table_width)

    # Print total nutrition summary
    print("Total Nutrition Summary")
    print("Calories: {}, Protein: {}g, Carbs: {}g, Fats: {}g".format(
        total_calories, total_protein, total_carbs, total_fats))

def createNutritionPlan_main(user_profile):
    print_nutrition(user_profile)
    b_list, ld_list, s_list = create_nutrition_plan(user_profile.calories, user_profile.macros.protein_grams, user_profile.macros.carbs_grams, user_profile.macros.fats_grams)
    l_list = [ld_list[x] for x in range(len(ld_list)) if x % 2 == 0]
    d_list = [ld_list[x] for x in range(len(ld_list)) if x % 2 == 1]
    b_list = get_recipe_info(b_list)
    l_list = get_recipe_info(l_list)
    d_list = get_recipe_info(d_list)
    s_list = get_recipe_info(s_list)
    b_list_extended = multi_servings(b_list)
    l_list_extended = multi_servings(l_list)
    d_list_extended = multi_servings(d_list)
    s_list_extended = multi_servings(s_list)

    chosen_meals = create_lp_model(user_profile, b_list_extended, l_list_extended, d_list_extended, s_list_extended)
    pretty_print_createNutritionPlan(chosen_meals)


# CreateWorkoutPlan

In [17]:


exercise_data_path = r"/content/drive/MyDrive/23-24/CS 4701/exercise model/exercisemodeldata.csv"

import pandas as pd
import numpy as np


def create_workout_plan(file_path, muscle_groups, time_limit=90):
    df = pd.read_csv(file_path)
    df['Muscle Group Sum'] = df[muscle_groups].sum(axis=1)
    df['PPM'] = df['Muscle Group Sum'] / df['Time Requirement (mins)']

    dict = {}
    for i in range(len(muscle_groups)):
        muscle = muscle_groups[i]
        dict[muscle] = 0

    df_filtered = df[df['Muscle Group Sum'] > 0]
    df_sorted = df_filtered.sort_values(by=['Muscle Group Sum', 'PPM'], ascending=[False, False])
    df_sorted = df_sorted
    selected_exercises = []
    total_time = 0
    switch = False

    while (total_time < time_limit) :
        vals = dict.values()
        maxval = max(vals)
        minval = min(vals)

        if 0 in vals and maxval > 0 and not switch:

            zero_groups = []
            nonzero_groups = []
            for k, v in dict.items():
                if v == 0:
                    zero_groups.append(k)
                else:
                    nonzero_groups.append(k)

            zero_groups_df = df_filtered[df_filtered[nonzero_groups].eq(0).all(axis=1)]
            zero_groups_df = zero_groups_df[zero_groups_df['Time Requirement (mins)'] <= (time_limit - total_time)]
            zero_groups_df["Zero Groups Sum"] = zero_groups_df[zero_groups].sum(axis=1)
            zero_groups_df = zero_groups_df.sort_values(by=['Zero Groups Sum', 'PPM'], ascending=[False, False])

            is_empty = zero_groups_df.empty

            if is_empty:
                switch = True
                continue

            chosen_row = zero_groups_df.iloc[0]
            selected_exercises.append(chosen_row['Exercise'])
            total_time += chosen_row['Time Requirement (mins)']

            for i in range(len(muscle_groups)):
                muscle = muscle_groups[i]
                muscle_points = chosen_row[muscle]
                dict[muscle] = dict[muscle] + muscle_points

            chosen_exercise = chosen_row['Exercise']
            df_filtered = df_filtered[df_filtered['Exercise'] != chosen_exercise]

        else:

            normal = df_filtered[df_filtered['Time Requirement (mins)'] <= (time_limit - total_time)]
            normal = normal.sort_values(by=['Muscle Group Sum', 'PPM'], ascending=[False, False])

            is_empty = normal.empty

            if is_empty:
                break

            chosen_row = normal.iloc[0]
            selected_exercises.append(chosen_row['Exercise'])
            total_time += chosen_row['Time Requirement (mins)']

            for i in range(len(muscle_groups)):
                muscle = muscle_groups[i]
                muscle_points = chosen_row[muscle]
                dict[muscle] = dict[muscle] + muscle_points

            chosen_exercise = chosen_row['Exercise']
            df_filtered = df_filtered[df_filtered['Exercise'] != chosen_exercise]



    pretty_string = ''
    for i in range(len(selected_exercises)):
        if i == len(selected_exercises)-1 :
            pretty_string = pretty_string + 'and '
        pretty_string = pretty_string + selected_exercises[i] + ', '

    pretty_string = pretty_string[0:-2]

    return "Your workout consists of " + pretty_string + ". It should take you approximately " + str(total_time) + " minutes.", selected_exercises, total_time



# muscle_groups_example = ["Triceps", "Biceps", "Legs"]
# time_limit_example = 30
# file_path = "exercisemodeldata.csv"
# print(create_workout_plan(file_path, muscle_groups_example, time_limit_example))






Chest = "Chest"
Triceps = "Triceps"
Back = "Back"
Shoulders = "Shoulders"
Biceps = "Biceps"
Legs = "Legs"
Core = "Core"
Arms = "Arms"
Push = "Push"
Pull = "Pull"
arms_exercises = [Biceps, Triceps]
push_exercises = [Chest, Triceps, Shoulders]
pull_exercises = [Back, Biceps]

matching_muscles = {
    Chest: ["Chest", "Pecs", "Pectorals"],
    Triceps: ["Tricep", "Tris"],
    Back: ["Back", "Lats"],
    Shoulders: ["Shoulders", "Delts", "Deltoids"],
    Biceps: ["Biceps"],
    Legs: ["Legs", "Quads", "Hamstring", "Calves"],
    Core: ["Core", "Abs"],
    Arms: ["Arms"],
    Push: ["Push"],
    Pull: ["Pull"],
}

def find_closest_muscles(muscles, muscles_mapping, threshhold=0.8):
    closest_muscles = set()  # Use a set to avoid duplicate entries
    for muscle in muscles:
      best_match_muscle = None
      highest_similarity = 0
      for muscle_enum, expressions in muscles_mapping.items():
          for expression in expressions:
              similarity = Levenshtein.ratio(muscle.lower(), expression.lower())
              if similarity > highest_similarity:
                  highest_similarity = similarity
                  best_match_muscle = muscle_enum
      if highest_similarity >= threshhold:
          if best_match_muscle == Arms:
              for e in arms_exercises:
                  closest_muscles.add(e)
          elif best_match_muscle == Pull:
              for e in pull_exercises:
                  closest_muscles.add(e)
          elif best_match_muscle == Push:
              for e in push_exercises:
                  closest_muscles.add(e)
          else:
            closest_muscles.add(best_match_muscle)
    if closest_muscles == []:
        return None
    return list(closest_muscles)

Minutes = "minutes"
Hours = "hours"
time_mapping = {
    Minutes: ["Minutes", "min", "mins", "m"],
    Hours: ["Hours", "hrs", "hr"]
}

def print_workout_plan(exercises, total_time):
    header = "Your Workout Plan"
    separator = "-" * len(header)
    formatted_exercises = "\n".join(exercises)

    print(f"{separator}\n{header}\n{separator}\n{formatted_exercises}\nTotal Time: {total_time} minutes\n{separator}")

def print_workout_details(goal):
    workout_goals = {
        1: "Weight Loss",
        2: "Maintain Weight",
        3: "Gain Muscle"
    }

    sets = ""
    reps = ""
    rest = ""

    if goal == 1:  # Weight Loss
        sets = "3-4 sets"
        reps = "10-15 reps or more"
        rest = "30-60 seconds rest"
    elif goal == 2:  # Maintain Weight
        sets = "3-4 sets"
        reps = "8-12 reps"
        rest = "60-90 seconds rest"
    else:  # Gain Muscle
        sets = "3-5 sets"
        reps = "6-12 reps"
        rest = "90-120 seconds rest"

    # Pretty print the workout details
    print(f"Workout Goal: {workout_goals.get(goal, 'Unknown Goal')}")
    print(f"Recommended Sets: {sets}")
    print(f"Recommended Reps: {reps}")
    print(f"Recommended Rest Periods: {rest}")

def print_ppl(user_profile):
    print("We've designed a 3 day workout routine for you!")

    # Day 1: Push Day
    plan, exercises, time = create_workout_plan(exercise_data_path, push_exercises)
    print("\nDay 1: Push Day (Chest, Triceps, Shoulders)")
    print_workout_plan(exercises, time)

    # Day 2: Pull Day
    plan, exercises, time = create_workout_plan(exercise_data_path, pull_exercises)
    print("\nDay 2: Pull Day (Back and Biceps)")
    print_workout_plan(exercises, time)

    # Day 3: Legs and Core Day
    plan, exercises, time = create_workout_plan(exercise_data_path, ["Legs", "Core"])
    print("\nDay 3: Legs and Core Day")
    print_workout_plan(exercises, time)
    print_workout_details(user_profile.goal)
    if user_profile.goal == 1:
        print("\nAdditional Recommendation: 30 minutes of cardio (running, swimming, biking, etc.)")
    if user_profile.goal == 2:
        print("\nAdditional Recommendation: 15 minutes of cardio (running, swimming, biking, etc.)")





def createWorkoutPlan_main(muscles, number, time, user_profile):
  chosen_muscles = find_closest_muscles(muscles, matching_muscles)
  time_limit = 90

  if number != "" and time != "":
    time_unit = find_closest_weight_unit(time, time_mapping)
    if time_unit:
      if (time_unit == Minutes):
        time_limit = float(number)
      else:
        time_limit = float(number)*60
  if chosen_muscles == []:
        print_ppl(user_profile)
  else:
      plan, exercises, time = create_workout_plan(exercise_data_path, chosen_muscles, time_limit)
      print_workout_plan(exercises, time)
      print_workout_details(user_profile.goal)

def extract_muscles_entity(cleaned_predictions):
  muscles_entities = []
  for token, label in cleaned_predictions:
    if label in ['B-Muscle', 'I-Muscle']:
      muscles_entities.append(token)
  return muscles_entities

def extract_number_and_time(cleaned_predictions):
  number_entity = ""
  time_entity = ""
  for i, (token, label) in enumerate(cleaned_predictions):
      if label in ['B-Number', 'I-Number']:
          if token.replace('.', '', 1).isdigit():
              number_entity = token
          else:
            try:
                number_entity = str(w2n.word_to_num(token.lower()))
            except ValueError:
                print("setting number_entity to", token)
                number_entity = token
      elif label in ['B-Time', 'I-Time']:
          time_entity = token
          break
  #print("returning", number_entity, time_entity)
  return number_entity, time_entity





# RecommendMeal

In [11]:
def extract_mealtype_entity(cleaned_predictions):
  mealtype_entity = []
  for token, label in cleaned_predictions:
    if label in ['B-MealType', 'I-MealType']:
      mealtype_entity.append(token)
  return mealtype_entity
Breakfast = "Breakfast"
Lunch = "Lunch"
Dinner = "Dinner"
Snack = "Snack"
mealtype_mapping = {
    Breakfast: ["Breakfast", "bfast", "break fast", "morning"],
    Lunch: ["Lunch", "Afternoon", "Noon"],
    Dinner: ["Dinner", "Evening"],
    Snack: ["Snack", "snacks", "quick bite"]
}
def pretty_print_recommendation(recommended_meals, meal_type):
    table_width = 140
    print(f"\n{meal_type} Recommendations (One serving)")
    print("-" * table_width)
    print("{:<50} {:<12} {:<12} {:<10} {:<10} {:<10} {:<10}".format(
        "Recipe", "Time (mins)", "Calories", "Protein (g)", "Fats (g)", "Carbs (g)", "Recipe Link"))
    print("-" * table_width)

    for meal in recommended_meals:
        name = meal[1]
        if len(name) > 47:
            name = name[:47] + '...'

        time, calories, protein, fats, carbs, link = meal[2], meal[4], meal[5], meal[6], meal[7], meal[9]

        print("{:<50} {:<12} {:<12} {:<10} {:<10} {:<10} {:<10}".format(
              name, time, calories, protein, fats, carbs, link))

    print("-" * table_width)

def recommendMeal_main(mealtype_entity, user_profile):
  chosen_mealtype = []
  for mte in mealtype_entity:
    result = find_closest_weight_unit(mte, mealtype_mapping)
    if result:
      chosen_mealtype.append(result)
  if chosen_mealtype != []:
    b_list, ld_list, s_list = create_nutrition_plan(user_profile.calories, user_profile.macros.protein_grams, user_profile.macros.carbs_grams, user_profile.macros.fats_grams)
    l_list = [ld_list[x] for x in range(len(ld_list)) if x % 2 == 0]
    d_list = [ld_list[x] for x in range(len(ld_list)) if x % 2 == 1]
    b_list = get_recipe_info(b_list)
    l_list = get_recipe_info(l_list)
    d_list = get_recipe_info(d_list)
    s_list = get_recipe_info(s_list)
    print("Consider having more than one serving for some of these:")
    for chosen_meal in chosen_mealtype:
      if chosen_meal == Breakfast:
        pretty_print_recommendation(b_list, Breakfast)
      elif chosen_meal == Lunch:
        pretty_print_recommendation(l_list, Lunch)
      elif chosen_meal == Dinner:
        pretty_print_recommendation(d_list, Dinner)
      elif chosen_meal == Snack:
        pretty_print_recommendation(s_list, Snack)

  else:
    createNutritionPlan_main(user_profile)



# RecommendExercise

In [12]:
def recommendExercise_main(muscle_groups):
    chosen_muscles = find_closest_muscles(muscle_groups, matching_muscles)
    capitalized_muscle_groups = [muscle.capitalize() for muscle in chosen_muscles]
    for muscle_group in capitalized_muscle_groups:
        exercises = create_workout_plan(exercise_data_path, [muscle_group])[1]

        # Header for each muscle group
        header = f"Exercises for {muscle_group}:"
        print("-" * len(header))
        print(header)
        print("-" * len(header))

        # Printing each exercise
        for exercise in exercises:
            print(f"{exercise}")

        # Ending line separator
        print("-" * len(header))
        print()

# Query UserInfo

In [13]:
class Macros:
    def __init__(self, calories, protein_percentage, carbs_percentage, fats_percentage):
        self.calories = calories
        self.protein_percentage = protein_percentage
        self.carbs_percentage = carbs_percentage
        self.fats_percentage = fats_percentage
        self.protein_grams = self.calculate_macros('protein')
        self.carbs_grams = self.calculate_macros('carbs')
        self.fats_grams = self.calculate_macros('fats')

    def calculate_macros(self, macro_type):
        if macro_type == 'protein':
            return (self.calories * self.protein_percentage) / 4
        elif macro_type == 'carbs':
            return (self.calories * self.carbs_percentage) / 4
        elif macro_type == 'fats':
            return (self.calories * self.fats_percentage) / 9
        else:
            return 0

    def display_macros(self):
        print(f"Daily Macros: \nProtein: {self.protein_grams:.1f}g, Carbs: {self.carbs_grams:.1f}g, Fats: {self.fats_grams:.1f}g")


class UserInfo:
    def __init__(self, height, weight, age, gender, goal, activity_level):
        self.height = height
        self.weight = weight
        self.age = age
        self.gender = gender
        self.goal = goal
        self.activity_level = activity_level
        TDEE = 0
        if gender == "M":
            TDEE = (10 * weight) + (6.25 * height) - (5 * age) + 5
        elif gender == "F":
            TDEE = (10 * weight) + (6.25 * height) - (5 * age) - 161
        goal_factor = [0.8, 1, 1.15]
        activity_factor = [1.2, 1.375, 1.55, 1.725, 1.9]
        TDEE *= goal_factor[self.goal-1]
        self.calories = TDEE * activity_factor[activity_level - 1]
        self.macros = self.set_macros()

    def set_macros(self):l
        if self.goal == 1:  # Lose Weight
            return Macros(self.calories, 0.30, 0.40, 0.30)
        elif self.goal == 2:  # Maintain Weight
            return Macros(self.calories, 0.25, 0.50, 0.25)
        elif self.goal == 3:  # Gain Muscle
            return Macros(self.calories, 0.35, 0.35, 0.30)

    @staticmethod
    def get_user_input():
        print("Please enter the following details:")
        height = input("Height (in cm): ")
        if height == "test":
          return UserInfo(180, 80, 21, "M", 2, 3)
        height =float(height)
        weight = float(input("Weight (in kg): "))
        age = int(input("Age: "))
        gender = input("Gender (M/F): ").upper()
        while gender not in ["M", "F"]:
            print("Invalid gender. Please enter 'M' for Male or 'F' for Female.")
            gender = input("Gender (M/F): ").upper()

        print("\nPlease select your fitness goal:")
        print("1. Lose Weight")
        print("2. Maintain Weight")
        print("3. Gain Muscle")
        goal = int(input("Your goal (1/2/3): "))
        while goal not in [1, 2, 3]:
            print("Invalid selection. Please enter 1, 2, or 3.")
            goal = int(input("Your goal (1/2/3): "))

        print("\nPlease select your activity level:")
        print("1. Sedentary (little or no exercise)")
        print("2. Lightly active (light exercise/sports 1-3 days a week)")
        print("3. Moderately active (moderate exercise/sports 3-5 days a week)")
        print("4. Very active (hard exercise/sports 6-7 days a week)")
        print("5. Super active (very hard exercise & physical job or training twice a day)")
        activity_level = int(input("Your activity level (1/2/3/4/5): "))
        while activity_level not in [1, 2, 3, 4, 5]:
            print("Invalid selection. Please enter a number from 1 to 5.")
            activity_level = int(input("Your activity level (1/2/3/4/5): "))

        return UserInfo(height,
                        weight,
                        age,
                        gender,
                        goal,
                        activity_level)


    def display_details(self):
        print("\nYour details:")
        print(f"Height: {self.height} cm")
        print(f"Weight: {self.weight} kg")
        print(f"Age: {self.age}")
        print(f"Gender: {'Male' if self.gender == 'M' else 'Female'}")
        goals = ["Lose Weight", "Maintain Weight", "Gain Muscle"]
        print(f"Fitness Goal: {goals[self.goal - 1]}")
        activity_levels = [
            "Sedentary",
            "Lightly active",
            "Moderately active",
            "Very active",
            "Super active"
        ]
        print(f"Activity Level: {activity_levels[self.activity_level - 1]}")
        print(f"Daily Calories: {self.calories}")
        self.macros.display_macros()
    def get_nutrition(self):
      return self.calories, self.macros

def userInfo_main():
    user_info = UserInfo.get_user_input()
    #user_info.display_details()
    return user_info



# Process

Before running this code vvv make sure to run all of the other modules first!

In [14]:
def process(intent, entities, user_profile):
  if (intent == "CreateNutritionPlan"):
      createNutritionPlan_main(user_profile)
  elif (intent == "CreateWorkoutPlan"):
      muscles_entity = extract_muscles_entity(entities)
      number_entity, time_entity = extract_number_and_time(entities)
      createWorkoutPlan_main(muscles_entity, number_entity, time_entity, user_profile)
  elif (intent == "QueryFood"):
      food_entity = extract_food_entity(entities)
      nutrient_entity = extract_nutrient_entity(entities)
      number_entity, weight_entity = extract_number_and_weight(entities)
      queryFood_main(food_entity, nutrient_entity, number_entity, weight_entity, user_profile)
  elif (intent == "QueryExercise"):
    workout_entity = extract_workout_entity(entities)
    query_exercise(workout_entity)
  elif (intent == "RecommendMeal"):
    mealtype_entity = extract_mealtype_entity(entities)
    recommendMeal_main(mealtype_entity, user_profile)
  elif (intent == "RecommendExercise"):
    muscles_entity = extract_muscles_entity(entities)
    recommendExercise_main(muscles_entity)
  else :
      print("I don't have the capabilities to answer that!")
def chatBot_main(user_profile):
  while True:
    print()
    user_input = input("Type your question or 'QUIT' to exit: ")
    print()
    if user_input.upper() == "QUIT":
        break

    # Predict Intent
    intent = predict_intent(user_input, intent_model, intent_tokenizer)
    #print(f"Predicted Intent: {intent}")

    # Predict Entities
    entities = predict_entities(user_input, NER_model, NER_tokenizer, ner_label_map)
    cleaned_entities = clean_predictions(entities)
    aligned_entities = align_words_with_entities(user_input, cleaned_entities)
    #print("Predicted Entities:", aligned_entities)

    process(intent, aligned_entities, user_profile)
  print("Exiting the program.")

# Run

In [ ]:
user_profile = userInfo_main()
chatBot_main(user_profile)